### Libs e  Funções ###

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import copy
import sys
from scipy import stats
import random

def save_image(data):
    return np.asarray(np.clip(data, 0, 255), dtype='uint8')


def incbri(img,percnt):
    nimg = img.copy();
    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(len(nimg[0][0])):
                value = (percnt/100)*nimg[i][j][k];
                nimg[i][j][k] = min(nimg[i][j][k] + value, 255);
    return nimg

def neg(img):
    nimg = img.copy();
    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(len(nimg[0][0])):
                nimg[i][j][k] = 255 - nimg[i][j][k];
    return nimg
#Histograma

def histg(img):
    nimg = img.copy();
    bgr = np.zeros(256*3);  
    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(len(nimg[0][0])):
                bgr[nimg[i][j][k] + (k*256)] += 1; 
    ag = open("global_hist.txt", "w+")
    bgr = save_image(bgr);
    ag.write(str(bgr))
    ag.close()
    return bgr

def histlocal(img):
    a = int(input("Entrada: ")) # Number of divisions in a format axa
    n, m = len(img)//a, len(img[0])//a # Size of image

    g = [[0 for i in range(256)] for j in range(a * a)]
    r = [[0 for i in range(256)] for j in range(a * a)]
    b = [[0 for i in range(256)] for j in range(a * a)]

    for i in range(len(img)):
        for j in range(len(img[i])):
            for k in range(len(img[i][j])):
                pos =  a * ((i//n) - ((i//n) >= a)) + ((j // m) - ((j//m) >= a))
                if k == 0:
                    g[pos][img[i][j][k]] += 1
                elif k == 1:
                    r[pos][img[i][j][k]] += 1
                else:
                    b[pos][img[i][j][k]] += 1
                hg, hr, hb = g[0], r[0], b[0]

    for i in range(1, len(g)):
        hg = hg + g[i]

    for i in range(1, len(r)):
        hr = hr + r[i]

    for i in range(1, len(b)):
        hb = hb + b[i]

    print(len(hg), len(hr), len(hb))
    rgb = r+g+b;
    ag = open("hist_local.txt", "w+")
    ag.write(str(rgb))
    ag.close()
    
    return

# Tranformadas

def explin(img,za,zb,zn,z1):
    nimg = img.copy();
    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(3):
                if(nimg[i][j][k] <= za):
                    nimg[i][j][k] = z1;
                elif(nimg[i][j][k] < zb):
                    nimg[i][j][k] = (((zn-z1)/(zb-za)) * (nimg[i][j][k]-za)) + z1;
                else:
                    nimg[i][j][k] = zn;    
                
    return nimg

def rlc(img):
    nimg = img.copy();
    his  = [0 for i in range(256)];

    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(3):
                his[nimg[i][j][k]] += 1;

    nhis = [x/(len(nimg)*(len(nimg[0]))*(len(nimg[0][0]))) for x in his];
    fhis  = [0.0 for i in range(256)];
    fhis[0] = nhis[0];
    for i in range(1,256):
        fhis[i] = fhis[i-1] + nhis[i];

    ifhist = [int(fhis[i]*255+0.5) for i in range(256)];

    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(3):
                nimg[i][j][k] = ifhist[nimg[i][j][k]];
    return nimg

def log(img):
    nimg = img.copy()
    for i in range(len(nimg)):
        for j in range(len(nimg[0])):
            for k in range(3):
                nimg[i][j][k] = 180*math.log((nimg[i][j][k]+1),10);
    return nimg

#Filtros
    
def mean3x3(img,dif):
    nimg = img.copy();
    for i in range(1,(len(nimg)-1)):
        for j in range(1,(len(nimg[0])-1)):
            for k in range(3):
                neig = nimg[i-1:i+2,j-1:j+2,k]
                if((abs(nimg[i][j][k]-neig) > dif).any()):
                    nimg[i][j][k] = np.mean(neig);
                    
    return nimg

def median3x3(img,dif):
    nimg = img.copy();
    for i in range(1,(len(nimg)-1)):
        for j in range(1,(len(nimg[0])-1)):
            for k in range(3):
                neig = nimg[i-1:i+2,j-1:j+2,k]
                if((abs(img[i][j][k]-neig) > dif).any()):
                    nimg[i][j][k] = np.median(neig);
                    
    return nimg

def minmax(img,dif,func):
    nimg = img.copy();
    for i in range(1,(len(nimg)-1)):
        for j in range(1,(len(nimg[0])-1)):
            for k in range(3):
                neig = img[i-1:i+2,j-1:j+2,k];
                if((abs(img[i][j][k]-neig) > dif).any()):
                    nimg[i][j][k] = func(np.concatenate(neig));

                   
    return nimg

def ruido(img, percent,ntype):
    nimg = img.copy();
    num = int(len(img)*len(img[0])*percent);
    while(num>0):
        i = random.randint(0,len(img)-1);
        j = random.randint(0,(len(img[0])-1));
        k = random.randint(0,len(img)-1);
        l = random.randint(0,(len(img[0])-1));
        if (noise[i][j] != ntype).all():
            if(ntype == 2):
                nimg[i][j] = 0;
                nimg[k][l] = 255;
            else:
                nimg[i][j] = ntype;
        num-=1
    return nimg


    
#Detecção de Bordas 

def color_quantization(img, K):
    Z = img.reshape((-1,3))
    Z = np.float32(Z)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    return res2

def robert_cross(img):
    gradient = lambda i,j: np.sqrt(np.square(img[i][j]-img[i+1][j+1]) + np.square(img[i][j+1]-img[i+1][j]))
    for i in range(len(img)-1):
        for j in range(len(img[0])-1):
            img[i][j] = gradient(i,j)
    key = cv2.waitKey(0) & 0xFF
    return save_image(img)

#BIC

def bic(img):
    colors = {}
    img = color_quantization(img, 64)
    color = lambda i, j: int(img[i][j][0]) + 4 * int(img[i][j][1]) + 16 * int(img[i][j][2]) #bgr
    limiar = lambda i, j: True if color(i,j) == color(i+1,j) == color(i+1, j+1) == color(i,j+1) else False
    histI, histB = np.zeros(64), np.zeros(64)
    for i in range(len(img) - 1):
        for j in range(len(img[0]) - 1):
            c = color(i,j)
            if not c in colors:
                colors[c] = len(colors)
            if limiar(i,j):
                histI[colors[c]] += 1
            else:
                histB[colors[c]] += 1
    hitzao = np.concatenate(histI,histB);
    return hitzao
    


# Questão 1) #

In [3]:
img = cv2.imread('cod.webp');
nimg = incbri(img,30)
cv2.imwrite('cod_bri+.webp',nimg)

True

# Questão 2)

In [4]:
img = cv2.imread('cod.webp');
nimg = neg(img)
cv2.imwrite('cod_neg.webp',nimg)

True

# Questão 3)#

In [18]:
img = cv2.imread('cod.webp');
hist = histg(img)

# Questão 4) #

In [16]:
img = cv2.imread('cod.webp');
hist = histlocal(img);

Entrada: 3
2304 2304 2304


# Questão 5) #

In [10]:
img = cv2.imread('test.png');
nimg = explin(img,190,230,255,0)
cv2.imwrite('test_expansão.png',nimg)

nimg2 = rlc(img)
cv2.imwrite('test_rlc.png',nimg2)

nimg3 = log(img)
cv2.imwrite('test_log.png',nimg3)

True

# Questão 6) #

In [19]:
img = cv2.imread('lena.jpg');
saltimage = ruido(img,0.02,255);
pepperimage = ruido(img,0.02,0);
noise = ruido(img,0.02,2)
cv2.imwrite('salt20.jpg',saltimage)
cv2.imwrite('pepper20.jpg',pepperimage)
cv2.imwrite('noise20.jpg',noise)



cv2.imwrite('lena_salt_mean3x3.jpg',mean3x3(saltimage,50))

cv2.imwrite('lena_pepper_mean3x3.jpg',mean3x3(pepperimage,25))

cv2.imwrite('lena_salt_median3x3.jpg',median3x3(saltimage,50))

cv2.imwrite('lena_pepper_median3x3.jpg',median3x3(pepperimage,25))

cv2.imwrite('lena_salt_min3x3.jpg',minmax(saltimage,50,min))

cv2.imwrite('lena_pepper_max3x3.jpg',minmax(pepperimage,25,max))


True

# Questão 7) #

In [6]:
img = cv2.imread('lena.jpg');
nimg = color_quantization(img, 64);
resimg = robert_cross(nimg);
cv2.imwrite('lena_border_detection.jpg',resimg);

# Questão 8) #

In [7]:
res2 = bic(nimg);
ag = open("bic.txt", "w+")
ag.write(str(res2))
ag.close()

TypeError: only integer scalar arrays can be converted to a scalar index